<a href="https://colab.research.google.com/github/guscldns/README_example_Source/blob/main/%EC%9E%A5%EA%B8%B0/polyglot_qlora_vicuna_%EB%94%B0%EB%9D%BC%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


https://github.com/jwj7140/polyglot_qlora_vicuna


# `transformers` meets `bitsandbytes` for democratzing Large Language Models (LLMs) through 4bit quantization

<center>
<img src="https://github.com/huggingface/blog/blob/main/assets/96_hf_bitsandbytes_integration/Thumbnail_blue.png?raw=true" alt="drawing" width="700" class="center"/>
</center>

Welcome to this notebook that goes through the recent `bitsandbytes` integration that includes the work from XXX that introduces no performance degradation 4bit quantization techniques, for democratizing LLMs inference and training.

In this notebook, we will learn together how to load a large model in 4bit (`gpt-neo-x-20b`) and train it using Google Colab and PEFT library from Hugging Face 🤗.

[In the general usage notebook](https://colab.research.google.com/drive/1ge2F1QSK8Q7h0hn3YKuBCOAS0bK8E0wf?usp=sharing), you can learn how to propely load a model in 4bit with all its variants.

If you liked the previous work for integrating [*LLM.int8*](https://arxiv.org/abs/2208.07339), you can have a look at the [introduction blogpost](https://huggingface.co/blog/hf-bitsandbytes-integration) to lean more about that quantization method.


In [ ]:
!nvidia-smi

Sun Jun 18 02:01:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

First let's load the model we are going to use - GPT-neo-x-20B! Note that the model itself is around 40GB in half precision

In [2]:
from datasets import load_dataset

data = load_dataset("changpt/ko-lima-vicuna", data_files="ko_lima_vicuna.json")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
data

DatasetDict({
    train: Dataset({
        features: ['conversations', 'id'],
        num_rows: 866
    })
})

In [4]:
data = data.map(
    lambda x: {
        'text': "\n".join([f"{'인간' if line['from']=='human' else '비서'}: {line['value']}{'</끝>' if line['from']!='human' else ''}" for line in x['conversations']])
      }
)

Map:   0%|          | 0/866 [00:00<?, ? examples/s]

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/polyglot-ko-12.8b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

Then we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

In [6]:
data = data.map(lambda samples: tokenizer(samples["text"],max_length=2048), batched=True)

Map:   0%|          | 0/866 [00:00<?, ? examples/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
data['train'][260]['text']

'인간: 20문제 게임을 해볼까요? 제가 생각하는 동물이 있어요.\n비서: 20문제 게임 좋아요!\n\n1. 포유류인가요?</끝>\n인간: 아니요\n비서: 2. 물 속에서 살고 있나요?</끝>\n인간: 아니요\n비서: 3. 파충류인가요?</끝>\n인간: 네\n비서: 4. 다리가 4개인가요?</끝>\n인간: 네\n비서: 5. 포식자인가요?</끝>\n인간: 네\n비서: 6. 길이가 1미터(3피트)보다 큰가요?</끝>\n인간: 네\n비서: 7. 아프리카에 서식하는가요?</끝>\n인간: 아니요\n비서: 8. 아시아에 서식하는가요?</끝>\n인간: 네\n비서: 9. 코모도 드래곤인가요?</끝>\n인간: 맞아요!\n비서: 만세! 9번의 질문으로 당신이 생각한 동물을 맞췄어요: 코모도 드래곤.\n\n재미있는 사실: 코모도 드래곤은 현재 가장 큰 도마뱀입니다!</끝>'

In [7]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [8]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [9]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6553600 || all params: 6608701440 || trainable%: 0.09916622894073424


Let's load a common dataset, english quotes, to fine tune our model on famous quotes.

In [10]:
tokenNum_ai = 12266     # "비서"
tokenNum_human = 8301   # "인간"
tokenNum_com = 29        # ":"

In [ ]:
!nvidia-smi

Sun Jun 18 02:09:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    27W /  70W |   9375MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
import transformers
from transformers import Trainer
import numpy as np

class maskTrainer(Trainer):
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)

  def compute_loss(self, model, inputs, return_outputs=False):
    # maskIndex_ai = torch.eq(inputs['input_ids'], torch.tensor(tokenNum_ai)).nonzero()
    # maskIndex_human = torch.eq(inputs['input_ids'], torch.tensor(tokenNum_human)).nonzero()
    # print(maskIndex_ai)

    for x in range(len(inputs['labels'])):
      # maskIndex_ai = torch.eq(inputs['input_ids'], torch.tensor(tokenNum_ai)).nonzero()
      # maskIndex_human = torch.eq(inputs['input_ids'], torch.tensor(tokenNum_human)).nonzero()
      # print(maskIndex_ai)

      maskindex1 = (inputs['labels'][x]==tokenNum_human).nonzero()[:, 0]
      temp = 0
      for i, index in enumerate(maskindex1):
        if (inputs['labels'][x][index+1] != tokenNum_com):
          maskindex1 = np.delete(maskindex1, i-temp)
          temp += 1

      maskindex2 = (inputs['labels'][x]==tokenNum_ai).nonzero()[:, 0]
      temp = 0
      for i, index in enumerate(maskindex2):
        if (inputs['labels'][x][index+1] != tokenNum_com):
          maskindex2 = np.delete(maskindex2, i-temp)
          temp += 1

      for i in range(len(maskindex1)):
        ai_index = -1
        for num in maskindex2:
          if (maskindex1[i] < num):
            ai_index = num
            break
        if (ai_index == -1):
          inputs['labels'][x][maskindex1[i]+2:] = -100
        else:
          inputs['labels'][x][maskindex1[i]+2:ai_index+2] = -100

    # print(inputs['labels'][x])

    outputs = model(**inputs)

    loss = outputs['loss']

    return (loss,outputs) if return_outputs else loss

Run the cell below to run the training! For the sake of the demo, we just ran it for few steps just to showcase how to use this integration with existing tools on the HF ecosystem.

In [ ]:
# import transformers

# # needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = maskTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        # warmup_steps=200,
        max_steps=400, ## 초소형만 학습: 10 step = 20개 샘플만 학습.
        fp16=True,
        output_dir="outputs",
        logging_steps=100,
        # num_train_epochs = 4,
        learning_rate=5e-4,

        lr_scheduler_type= "cosine",
        #optim="paged_adamw_8bit"

    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()



You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


In [ ]:
print("wow")

wow


In [ ]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [ ]:
model.save_pretrained("./saved")

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(stop):])).item():
                return True

        return False

stop_words = ["</끝>"]
stop_words_ids = [tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze() for stop_word in stop_words]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

In [15]:
def gen(x):
    prompt = f"인간: {x}\n비서:"
    gened = model.generate(
        **tokenizer(
            prompt,
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=512,
        temperature=0.8,
        # no_repeat_ngram_size=3,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
        stopping_criteria=stopping_criteria
    )
    return tokenizer.decode(gened[0])

In [ ]:
gen('슈카월드가 무엇인가요? 자세히 설명 부탁합니다')

In [ ]:
gen('공공성을 측정하기위한 방법은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
gen('주식 시장에서 안정적으로 수익을 얻기 위한 방법은?')

In [ ]:
gen('풋옵션과 콜옵션의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?')

In [ ]:
gen('풋옵션 매도와 콜옵션 매수의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?')

In [ ]:
gen("마진콜이 발생하는 이유가 뭐야? 그리고 어떻게 해야 마진콜을 막을 수 있어?")